In [3]:
using IJulia
installkernel("julia_ITensors","--sysimage=~/.julia/sysimages/sys_itensors.so")

[ Info: Installing julia_ITensors kernelspec in /Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9


"/Users/mschuylerm/Library/Jupyter/kernels/julia_itensors-1.9"

In [4]:
using ITensors
using ITensors.HDF5
using DataFrames

# Description

In this notebook I run DMRG for multiple 1D spin systems. I look at spin chain lengths of $L = 10, 100$ for the Ising Model, the Transverse Field Ising Model, and the Heisenberg Model. I am working on the J1-J2 Model. After running DMRG, I save the optimized MPS to this local directory and then copy it over to the appropriate directory in my mps-rnn repository as an 'init.hdf5' file that gets used for hierarchical initialization of an mps-rnn.

# Spin Chain

In [5]:
function spinchain_open(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end

    return interactions
end

spinchain_open (generic function with 1 method)

In [6]:
function spinchain_peri(L)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L])
    
    for spin in 1:L-1
        neighbor = spin+1
        push!(interactions[!,"$spin"],[spin,neighbor])
    end
    
    push!(interactions[!,"$L"],[10,1])
    
    return interactions
end

spinchain_peri (generic function with 1 method)

# Ising Model

In [5]:
function dmrg_1D_ising(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)/sanity_check/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    for i =1:L
        println("M",i)
        println(psi[i])
    end
    
    return energy_per_spin
end

dmrg_1D_ising (generic function with 1 method)

In [6]:
dmrg_1D_ising(10,-1.0,2,"open",true);


Running DMRG for a length 10 spin chain

After sweep 1 energy=-8.999996320485666  maxlinkdim=10 maxerr=8.23E-04 time=11.651
After sweep 2 energy=-9.000000000000007  maxlinkdim=2 maxerr=9.96E-11 time=0.027
After sweep 3 energy=-8.999999999999998  maxlinkdim=2 maxerr=4.37E-31 time=0.009
After sweep 4 energy=-8.999999999999996  maxlinkdim=2 maxerr=6.66E-16 time=0.008
After sweep 5 energy=-8.999999999999998  maxlinkdim=2 maxerr=1.39E-17 time=0.007
After sweep 6 energy=-8.999999999999996  maxlinkdim=2 maxerr=6.66E-16 time=0.008
After sweep 7 energy=-8.999999999999991  maxlinkdim=2 maxerr=6.66E-16 time=0.009
After sweep 8 energy=-9.0  maxlinkdim=2 maxerr=6.80E-16 time=0.007
After sweep 9 energy=-9.0  maxlinkdim=2 maxerr=6.80E-16 time=0.010
After sweep 10 energy=-9.0  maxlinkdim=2 maxerr=6.80E-16 time=0.031

A sample from the optimized MPS looks like:
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


LoadError: HDF5.API.H5Error: Error creating file ../out/ising_fm_1d_L10/sanity_check/init.hdf5
libhdf5 Stacktrace:
 [1] [0m[1mH5FD__sec2_open[22m: File accessibility/Unable to open file
[90m     unable to open file: name = '../out/ising_fm_1d_L10/sanity_check/init.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602[39m
  ⋮

# Transverse Field Ising Model

In [13]:
function dmrg_1D_tfim(L::Int, J, h, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
        os += 4*J,"Sz",i,"Sz",j
    end
    for k=1:L
        os += 2*h,"Sx",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setmindim!(sweeps,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/ising_fm_1d_L$(L)_h1/chi$maxbd/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
    
    println("")
    for i in 1:length(psi)
        @show psi[i]
    end
    
    return energy_per_spin
end

dmrg_1D_tfim (generic function with 1 method)

In [14]:
dmrg_1D_tfim(20,-1.,-1.,2,"open",true)


Running DMRG for a length 20 spin chain

After sweep 1 energy=-25.096148256345923  maxlinkdim=10 maxerr=3.56E-03 time=0.050
After sweep 2 energy=-25.10778212976226  maxlinkdim=17 maxerr=9.85E-11 time=0.046
After sweep 3 energy=-25.107797084232878  maxlinkdim=14 maxerr=8.39E-11 time=0.027
After sweep 4 energy=-25.107797108101682  maxlinkdim=10 maxerr=6.40E-11 time=0.037
After sweep 5 energy=-25.067283655731565  maxlinkdim=2 maxerr=8.69E-04 time=0.020
After sweep 6 energy=-25.068551096318153  maxlinkdim=2 maxerr=3.87E-04 time=0.031
After sweep 7 energy=-25.06861588256404  maxlinkdim=2 maxerr=3.69E-04 time=0.021
After sweep 8 energy=-25.06862006688664  maxlinkdim=2 maxerr=3.68E-04 time=0.040
After sweep 9 energy=-25.06862034639098  maxlinkdim=2 maxerr=3.68E-04 time=0.020
After sweep 10 energy=-25.06862036514857  maxlinkdim=2 maxerr=3.68E-04 time=0.022

A sample from the optimized MPS looks like:
[1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]

psi[i] = ITensor ord=2
Dim 1: (

-1.2534310182574284

# Heisenberg Model

In [18]:
function dmrg_1D_heis(L::Int, J, maxbd::Int, bc::String, save::Bool)
    
    if bc == "open"
        interactions = spinchain_open(L)
    elseif bc == "peri"
        interactions = spinchain_peri(L)
    end
    
    sites = siteinds("S=1/2",L)
    println("\nRunning DMRG for a length $L spin chain\n")
    
    # Build Hamiltonian
    os = OpSum()
    for k=1:ncol(interactions)
        i = interactions[!,k][1][1]
        j = interactions[!,k][1][2]
    os += 4*J,"Sz",j,"Sz",k
    os += 4*J/2,"S+",j,"S-",k
    os += 4*J/2,"S-",j,"S+",k
    end
    H = MPO(os,sites)

    psi0 = randomMPS(sites, 40)

    sweeps = Sweeps(10)
    setmaxdim!(sweeps, 10,20,100,100,maxbd)
    setcutoff!(sweeps, 1E-10)

    energy, psi = dmrg(H, psi0, sweeps, outputlevel=1)
    energy_per_spin = energy / L
    
    println("\nA sample from the optimized MPS looks like:\n",sample(psi))
    
    if save   
        f = h5open("../out/heis_afm_1d_L$(L)/init.hdf5","w") 
        write(f,"psi",psi)
        close(f)
    end
        
    return energy_per_spin
end

dmrg_1D_heis (generic function with 1 method)

In [22]:
dmrg_1D_heis(20,1.,5,"open",true)


Running DMRG for a length 20 spin chain

After sweep 1 energy=-34.682752881614775  maxlinkdim=10 maxerr=3.82E-03 time=0.056
After sweep 2 energy=-34.729858024171946  maxlinkdim=20 maxerr=8.63E-08 time=0.054
After sweep 3 energy=-34.729893326088515  maxlinkdim=39 maxerr=9.70E-11 time=0.151
After sweep 4 energy=-34.72989332364747  maxlinkdim=39 maxerr=9.99E-11 time=0.196
After sweep 5 energy=-34.643808881300146  maxlinkdim=5 maxerr=1.50E-03 time=0.032
After sweep 6 energy=-34.64444844437751  maxlinkdim=5 maxerr=8.97E-04 time=0.022
After sweep 7 energy=-34.644543173387035  maxlinkdim=5 maxerr=8.84E-04 time=0.033
After sweep 8 energy=-34.644563744742705  maxlinkdim=5 maxerr=8.80E-04 time=0.021
After sweep 9 energy=-34.64456875308565  maxlinkdim=5 maxerr=8.80E-04 time=0.037
After sweep 10 energy=-34.64457001072666  maxlinkdim=5 maxerr=8.79E-04 time=0.026

A sample from the optimized MPS looks like:
[2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2]


-1.732228500536333

# J1-J2 Model?

work on this later...

In [89]:
function spinchain_J1J2(L::Int,j1::Float64,j2::Float64)
    
    interactions = DataFrame(["$spin"=>[] for spin in 1:L-1])
    
    for spin in 1:L-1
        nneighbor = spin+1
        push!(interactions[!,"$spin"],[spin,nneighbor,j1])
    end

    for spin in 1:L-2
        nnneighbor = spin+1
        push!(interactions[!,"$spin"],[spin,nnneighbor,j2])
    end

    return interactions
end

spinchain_J1J2 (generic function with 2 methods)

In [90]:
spinchain_J1J2(10,1.,2.)

AssertionError: AssertionError: Data frame is corrupt: length of column :9 (2) does not match length of column 1 (3). The column vector has likely been resized unintentionally (either directly or because it is shared with another data frame).